In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [18]:
cmd="SELECT osm_id,contour_line_id FROM %s"%osm_nodes_table_name
cursor.execute(cmd)
nodes_data=pd.DataFrame(cursor.fetchall())

In [19]:
nodes_data

,osm_id,contour_line_id
0,1243805918,580
1,3510978185,580
2,3510978187,580
3,5880263938,580
4,10087242750,580
...,...,...
17430,301196406,115213
17431,294934188,115402
17432,294934059,115430
17433,10703146602,115480


In [4]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
    `osm_id` BIGINT NOT NULL,
    `contour_line_id` INT NOT NULL,
    `elevation` FLOAT,

    FOREIGN KEY (contour_line_id) REFERENCES %s(id),
    PRIMARY KEY (osm_id)
    );
        """%(osm_nodes_table_name,contours_lines_table_name)
    
    cursor.execute(cmd)



    
    cmd="""CREATE TABLE `%s`(
        `osm_begin` BIGINT NOT NULL,
        `osm_end` BIGINT NOT NULL,
        `osm_key` INT NOT NULL,
        `contour_line_id` INT NOT NULL,
        `edge_coordinate` FLOAT NOT NULL,
        `elevation` FLOAT NOT NULL,

        FOREIGN KEY (osm_begin) REFERENCES %s(osm_id),
        FOREIGN KEY (osm_end) REFERENCES %s(osm_id),
        FOREIGN KEY (contour_line_id) REFERENCES %s(id),
        PRIMARY KEY (osm_begin,osm_end,osm_key,edge_coordinate)
        

          )
          ENGINE=InnoDB;
        """%(osm_intersections_table_name,osm_nodes_table_name,osm_nodes_table_name,contours_lines_table_name)
    cursor.execute(cmd)
    

2

2